# 基于 Transformer 的英译中翻译模型训练实践

在前文中，我们已经实现了一个优化版本的 Transformer，这为我们接下来的训练实践奠定了坚实的基础。现在，我们将把目光聚焦于 Transformer 的训练过程，以典型的英译中翻译任务为案例，深入实践训练 Transformer 模型的整个流程。

推荐观看： [CSDN-理解Transformer](https://blog.csdn.net/weixin_44878336/article/details/142485944)

---

## 基础知识

---

### 1. 什么是分词器？

分词器（tokenizer）的任务是把自然语言文本分解成一个个离散的单元，比如单词、子词，甚至是字符。它的核心目标是让模型能“看懂”文本的基本结构。比如，面对“Hi, 世界你好”这样的句子，tokenizer 会把它拆成`Hi`、`,`、`世界`、`你好` 这样的 token，并为每个 token 分配一个唯一的 ID。这些 ID 是模型能处理的数字形式，但它们本身并没有语义信息，只是符号。如：

**原文：** `Hi, 世界你好` \
**分词：** `Hi = 12`、`, = 9`、`世界 = 63`、`你好 = 28` \
**序列：** `[12,9,63,28]`

### 2. 什么是词嵌入？

词嵌入（Word Embedding）的任务是把这些符号转化为语义化的向量。它通过嵌入矩阵，把每个 token 的 ID 映射到一个固定维度的向量空间中。比如，在一个 4 维的嵌入矩阵中（实际上使用的维度要大得多），“good”会被映射成一个 4 维的向量，这个向量不仅包含了“good”这个单词的语义信息，还能反映它与其他单词的关系，比如“good”和“nice”可能会在向量空间中更接近。如：

$$
E_{good} = \begin{bmatrix}
 0.23 \\
 0.26 \\
 0.66 \\
 0.43 \\
\end{bmatrix}
,
E_{nice} = \begin{bmatrix}
 0.19 \\
 0.25 \\
 0.60 \\
 0.46 \\
\end{bmatrix}
\Rightarrow
||E_{good} - E_{nice} || = 0.0787
$$

分词器和词嵌入的关系，就像是一场接力赛的两棒：一棒负责把文本拆解成有意义的“块”，另一棒则把这些“块”转化为能让模型理解的“语言”。它们的分工明确，但又紧密相连，共同决定了模型对文本的理解能力。

## 相关库引入

---

在本章我们不考虑如何训练一个 Tokenizer 而是直接引入 huggingface 的 `AutoTokenizer` 库。

**文档：** [huggingface-AutoTokenizer](https://huggingface.co/docs/transformers/v4.49.0/en/model_doc/auto#transformers.AutoTokenizer)

In [ ]:
import torch
import torch.nn.functional as F
from datasets import Dataset
import pandas as pd
from safetensors.torch import load_file
from pandarallel import pandarallel
from transformers import (
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

# 数据路径
data_path = './dataset/wmt_zh_en_training_corpus.csv'

# 测试标志
run_test_cell = False

### 1. 使用 Qwen-tokenizer

Qwen-tokenizer是Qwen模型的分词器，其采用字节级字节对编码（BBPE，Byte-Level Byte-Pair Encoding）。BBPE从UTF-8字节开始构建词汇表，初始词汇表包含所有256个可能的字节。然后，通过迭代合并训练数据中最频繁出现的字节对，形成更长的子词，直到词汇表达到预设的大小。例如，一个常见的中文字符可能被合并为一个token，而不是拆分为多个字节。

本地 `./tokenizer` 目录下存放了已经下载好的 Qwen-tokenizer ，可以直接使用。资源下载地址：[huggingface-Qwen-tokenizer](https://huggingface.co/Qwen/Qwen-tokenizer/tree/main)

In [ ]:
if run_test_cell:
    # 加载预训练模型的分词器
    tokenizer = AutoTokenizer.from_pretrained('./tokenizer/Qwen-tokenizer')

    # vocab 词表长度
    print(f'tokenizer vocab size: {tokenizer.vocab_size}')

    # 测试分词器
    test_text = "你好，世界！"
    print(f'\"{test_text}\" 的分词结果为：',tokenizer(test_text)['input_ids'])

### 2. Qwen-tokenizer 特殊 token

|                      token                      |  token ID   | 描述                                                 |
|:-----------------------------------------------:|:-----------:|:---------------------------------------------------|
|             <&#124;endoftext&#124;>             |   151643    | 表示文本结束的特殊字符，在 Qwen-7B 和 Qwen-7B-Chat 中用于标记文本的结束位置。 |
|             <&#124;im_start&#124;>              |   151644    | 在 Qwen-7B-Chat 中用于标记交互的开始。                         |
|              <&#124;im_end&#124;>               |   151645    | 在 Qwen-7B-Chat 中用于标记交互的结束。                         |
| <&#124;extra_0&#124;> ~ <&#124;extra_204&#124;> |      -      | 这些是备用的特殊字符，开发者可以根据需要使用它们来实现特定的功能或扩展模型的能力。          |

事实上上述特殊 token 都是给 Qwen-7B 及其相关系列模型使用的，我们不需要使用这些特殊 token ，对于接下来的训练我们需要自定义三个特殊的 token ：
- `<bos>` : 用来标记序列开始
- `<eos>` : 用来标记序列结束
- `<pad>` : 补全标记，将序列处理成特定长度

In [ ]:
custom_tokenizer = AutoTokenizer.from_pretrained(
    './tokenizer/Qwen-tokenizer',
    trust_remote_code=True,
    bos_token='<bos>',  # 151646
    eos_token='<eos>',  # 151647
    pad_token='<pad>',  # 151648
)

**测试特殊 token**

In [ ]:
if run_test_cell:

    # 增加其他特殊字符
    custom_tokenizer.add_tokens(["<think>"], special_tokens=True)

    # 未定义特殊字符的情况
    print(tokenizer("<bos>你好，世界<eos><pad><pad><pad><pad>")['input_ids'])

    # 定义特殊字符的情况
    print(custom_tokenizer("<bos>你好，世界<eos><pad><pad><pad><pad>")['input_ids'])

    sentences = [
        '<bos>你好，世界<eos>' ,
        '<bos>什么是变形金刚<eos>' ,
    ]

    print(custom_tokenizer.batch_encode_plus(sentences))
    print(custom_tokenizer.batch_encode_plus(
        sentences,
        max_length=30,
        padding="max_length",
        truncation=True
    )['input_ids'])

## 构建数据集

---

### 1. 下载中英翻译语料数据集

请将下载好的相关语料数据存放在本地 `./dataset` 目录下，资源下载地址：[魔塔-WMT中英机器翻译训练集](https://modelscope.cn/datasets/iic/WMT-Chinese-to-English-Machine-Translation-Training-Corpus/files)


In [ ]:
# 文件读取测试
if run_test_cell:
    df = pd.read_csv(data_path)

    print(df.head())  # 查看前几行数据
    print(df.describe())  # 数据统计描述

## 2. 处理语料数据集

处理数据语料需要按照 huggingface 的 `Dataset` 标准来处理，具体来说，就是一列的处理结果都要用字典的形式返回，如下面的 `data_preprocess(cols, tokenizer, dec_seq_len)` 函数。

In [ ]:
def data_preprocess(cols, tokenizer, dec_seq_len):
    """
    数据预处理

    :param cols: 传入的列
    :param tokenizer: 分词器
    :param dec_seq_len: 解码器序列长度
    :return: (英文序列，中文序列，预测输出序列)
    """

    cn_seqs = tokenizer.batch_encode_plus(
        cols['0'],
        max_length=dec_seq_len + 1,
        padding="max_length",
        truncation=True
    )

    cn_seqs_ids = cn_seqs['input_ids']
    cn_seqs_mask = cn_seqs['attention_mask']

    en_seqs = tokenizer.batch_encode_plus(
        cols['1'],
        max_length=dec_seq_len,
        padding="max_length",
        truncation=True
    )

    return {
        "input_seq": en_seqs['input_ids'],
        "input_mask": en_seqs['attention_mask'],
        "output_seq": [row[:dec_seq_len] for row in cn_seqs_ids],
        "output_mask": [row[:dec_seq_len] for row in cn_seqs_mask],
        "labels": [row[1:] for row in cn_seqs_ids],
    }

def load_dataset_from_csv(data_path, tokenizer, dec_seq_len, sample = None):
    """
    数据预处理

    :param data_path: 数据路径
    :param tokenizer: 分词器
    :param dec_seq_len: 解码器序列长度
    :param sample: 采样数量
    :return: (英文序列，中文序列，预测输出序列)
    """

    # 启用 pandarallel 并行
    pandarallel.initialize()

    # 读取并加载数据
    df = pd.read_csv(data_path)
    if sample is not None:
        df = df.sample(n=sample)

    # 对中文列去除字符串中的空格，并添加开始和结束的 token
    df['0'] = df['0'].parallel_apply(lambda x: f'{tokenizer.bos_token}{str(x).replace(" ", "")}{tokenizer.eos_token}')

    # 转换为 Dataset
    hf_dataset = Dataset.from_pandas(df)

    # 预处理数据
    hf_dataset = hf_dataset.map(
        lambda x : data_preprocess(x, tokenizer, dec_seq_len),
        batched=True,
        remove_columns=["0", "1"]
    )

    # 设置数据集格式和所需要的列
    hf_dataset.set_format(type='torch', columns=["input_seq", "output_seq", "input_mask", "output_mask", 'labels'])

    return hf_dataset

**测试数据集读取**

In [ ]:
if run_test_cell:

    dec_seq_len = 30

    # 通过 csv 加载
    dataset = load_dataset_from_csv(data_path, custom_tokenizer, dec_seq_len, sample=320_000)

    # 存储处理好的数据
    # dataset.save_to_disk('./dataset/wmt_320000')

    # 直接加载已经处理好的数据
    # dataset = Dataset.load_from_disk('./dataset/wmt_10000')

    # 测试
    data = dataset.__getitem__(10)
    en_seq = data['input_seq']
    en_seq_mask = data['input_mask']
    cn_seq = data['output_seq']
    cn_seq_mask = data['output_mask']
    labels = data['labels']

    print(f'en_seq size: {en_seq.size()}')
    print(f'en_seq_mask size: {en_seq_mask.size()}')
    print(f'cn_seq size: {cn_seq.size()}')
    print(f'cn_seq_mask size: {cn_seq_mask.size()}')
    print(f'labels size: {labels.size()}')

    print('en : ', custom_tokenizer.decode(en_seq.type(torch.int)))
    print('en mask', en_seq_mask)
    print('cn : ', custom_tokenizer.decode(cn_seq.type(torch.int)))
    print('cn mask', cn_seq_mask)
    print('pr : ', custom_tokenizer.decode(labels.type(torch.int)))

## 训练 Transformer

---

由于训练神经网络的具体实现方法都是大致相同的，区别点就是损失函数的定义，我们可以直接使用 huggingface 的 `transformers` 库提供的 `Trainer` 类来训练模型，该类可以通过设置参数或继承的方式来实现自定义损失函数，本次我们采用了定义了损失函数的方法： `compute_loss_func(outputs, labels, num_items_in_batch)` 。

参考文档：[huggingface-Trainer.compute_loss_func](https://huggingface.co/docs/transformers/v4.50.0/en/main_classes/trainer#transformers.Trainer.compute_loss_func)

### 1. 训练模型

In [ ]:
from transformer import Transformer

one_hot_len = custom_tokenizer.vocab_size + len(custom_tokenizer.all_special_tokens) + 3
pad_token_id = custom_tokenizer.encode(custom_tokenizer.pad_token)[0]

# 配置
# =====================================
## 模型参数
encoder_num = 6                             # 编码器数量
decoder_num = 6                             # 解码器数量
vocab_size = one_hot_len                    # 词表大小
dim_emb = 512                               # 词向量维度
dim_head = 64                               # 注意力头维度
head_num = 8                                # 注意力头数
continue_train = False                      # 是否继续之前的训练
weights_path = "./model/model.safetensors"  # 如果继续之前的训练，需要指定权重的位置

## 训练参数
training_args = TrainingArguments(
    output_dir="./results",            # 模型保存路径
    eval_strategy="epoch",             # 每个 epoch 进行评估
    save_strategy="best",              # 保存最优结果
    learning_rate=1e-4,                # 学习率
    per_device_train_batch_size=32,    # 每个设备上的训练批次大小
    per_device_eval_batch_size=64,     # 每个设备上的评估批次大小
    num_train_epochs=20,               # 训练轮数
    weight_decay=0.01,                 # 权重衰减
    logging_dir="./logs",              # 日志路径
    logging_steps=10,                  # 日志记录步数
    load_best_model_at_end=True,       # 训练结束时加载最优模型
    label_names=['labels'],            # 设置标签名
)
# =====================================

# 加载数据集
# dataset = load_dataset_from_csv(data_path, custom_tokenizer, dec_seq_len)
dataset = Dataset.load_from_disk("./dataset/wmt_10000")

# 按 80% 训练集和 20% 测试集的比例划分
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

# 模型定义
en_cn_translate_model = Transformer(encoder_num, decoder_num, vocab_size, dim_emb, dim_head, head_num)
en_cn_translate_model.training = True

# 是否继续之前的训练
if continue_train:
    # 直接加载为状态字典
    state_dict = load_file(weights_path)
    en_cn_translate_model.load_state_dict(state_dict)

# 定义损失函数
def compute_loss_func(outputs, labels, num_items_in_batch):

    # 测试损失
    if num_items_in_batch is None:
        output,_,_ = outputs

        # 对 softmax 输出取对数，得到 log-probs
        log_probs = torch.log(output)

        # 使用 NLLLoss 计算损失
        loss = F.nll_loss(
            log_probs.view(-1, log_probs.size(-1)),
            labels.view(-1),
            ignore_index=pad_token_id
        )
        return loss

    # 训练损失
    loss = F.cross_entropy(
        outputs.view(-1, outputs.size(-1)),
        labels.view(-1),
        ignore_index=pad_token_id
    )
    return loss

#设置训练器
trainer = Trainer(
    model=en_cn_translate_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_loss_func=compute_loss_func
)

trainer.train()
trainer.save_model("./model")

### 2. 测试模型

In [ ]:
from safetensors.torch import load_file
import torch
from transformers import AutoTokenizer
from transformer import Transformer

if run_test_cell:

    custom_tokenizer = AutoTokenizer.from_pretrained(
        './tokenizer/Qwen-tokenizer',
        trust_remote_code=True,
        bos_token='<bos>',  # 151646
        eos_token='<eos>',  # 151647
        pad_token='<pad>',  # 151648
    )
    one_hot_len = custom_tokenizer.vocab_size + len(custom_tokenizer.all_special_tokens) + 3

    # 设置配置
    encoder_num = 6           # 编码器数量
    decoder_num = 6           # 解码器数量
    vocab_size = one_hot_len  # 词表大小
    dim_emb = 512             # 词向量维度
    dim_head = 32             # 注意力头维度
    head_num = 16             # 注意力头数

    # 初始化空模型
    model = Transformer(encoder_num, decoder_num, vocab_size, dim_emb, dim_head, head_num)

    # 加载 .safetensors 权重文件
    weights_path = "./model/model.safetensors"

    # 直接加载为状态字典
    state_dict = load_file(weights_path)
    model.load_state_dict(state_dict)

    #####

    en_text = 'How are you'
    token = '<bos>'

    # 进行编码
    input_seq = custom_tokenizer.batch_encode_plus(
        [en_text], return_attention_mask = False
    )['input_ids']
    print(input_seq)
    input_seq = torch.tensor(input_seq)
    output_seq = torch.tensor(custom_tokenizer.encode(token)).view(1, -1)

    kv_caches = None
    enc_output_cache = None

    line = ''
    for _ in range(30):
        output, enc_output_cache ,kv_caches = model(
            input_seq = input_seq,
            output_seq = output_seq,
            enc_output_cache = enc_output_cache,
            dec_kv_caches = kv_caches
        )
        token_id = torch.argmax(output, dim = -1)
        output_seq = token_id.view(1, 1)
        token = custom_tokenizer.decode(token_id.view(-1))
        line += token
        if token == '<eos>':
            break

    print(line)